In [1]:
# if os.name != 'posix':
#     %matplotlib ipympl
# else:
#     %matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
# plt.rcParams.update({
#     "text.usetex": True,
#     'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
# })
import os
import h5py

print(os.getpid())
%cd ../

import pylib.mix as mix
import pylib.measurement as mse

10365
/media/work/docs/codes/QuCF/scripts-py


In [11]:
# -------------------------------------------------------------------------------
# --- Create a diagonal matrix with x-coordinates on the main diagonal ---
# -------------------------------------------------------------------------------
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info.operators import Operator
from qiskit.compiler import transpile
from scipy.linalg import expm
mix.reload_module(mix)

nq_ = 10
t_ = 2.0;  # time interval;
N_ = 1 << nq_

# x_min, x_max = 0.0, np.pi
# x_min, x_max = -np.pi, 0.0
x_min, x_max = -np.pi/2., np.pi/2.

x = np.linspace(x_min, x_max, N_) 
A = np.diag(x)

# --- Decompose the diagonal matrix into a tensor product of Z-Pauli gates ---
A_dec_ = mix.hermitian_to_Z_pauli(A, flag_filter = True, small_coef = 1e-14, flag_print_details=True)

---------------------------------------------------------------
--- Decomposition of a Hermitian into Pauli tensor products ---
nq:  10
Total number of possible combinations of (I,Z) Pauli matrices 1024
step: 1000
n. of Pauli products:  10
---------------------------------------------------------------


In [13]:
# -------------------------------------------------------
# --- Save the circuit for the exp(-1j * t_ * A_dec_) ---
# -------------------------------------------------------
path_root = "./tools/HS-diag-matrix/"
fname = "HS_n{:d}_xm{:d}.oracle".format(nq_, int(x[-1]))
fullname = path_root + "/" + fname

name_qr = "rx"  # name of the target register

print("Storing to: {:s}".format(fullname))
ff = open(fullname, "w")
counter_q = -1
for ii in range(len(A_dec_)):
    if list(A_dec_[ii][2]) == [0]*nq_:
        continue
    counter_q += 1
    angle_Rz = A_dec_[ii][0] * 2. * t_
    str_az = "{:0.12e}".format(angle_Rz)
    line_one = "gate Rz {:s}[{:d}] {:s} end_gate".format(name_qr, counter_q, str_az)
    ff.write(line_one + "\n")
    print("az: {:0.12e}".format(angle_Rz))
ff.close()

Storing to: ./tools/HS-diag-matrix//HS_n10_xm1.oracle
az: -6.141921121388e-03
az: -1.228384224278e-02
az: -2.456768448555e-02
az: -4.913536897110e-02
az: -9.827073794220e-02
az: -1.965414758844e-01
az: -3.930829517688e-01
az: -7.861659035376e-01
az: -1.572331807075e+00
az: -3.144663614150e+00


In [ ]:
az: -3.910068426197e-03
az: -7.820136852395e-03
az: -1.564027370479e-02
az: -3.128054740958e-02
az: -6.256109481916e-02
az: -1.251221896383e-01
az: -2.502443792766e-01
az: -5.004887585533e-01
az: -1.000977517107e+00
az: -2.001955034213e+00

In [9]:
# -------------------------------------------------------------
# --- TO RECHECK: Form a circuit to model exp(-1j * t_ * A_dec_) ---
# -------------------------------------------------------------
import pylib.QC as QC

# --- Construction of the circuit ---
qc_dec = QC.Circuit(nq_)
qc_dec.form_from_pauli(A_dec_, t_, sel_error_order = 2)

# draw the circuit:
print(qc_dec.form_qiskit_circuit().draw(fold = 160)); 
print("")

# get the matrix representation of the circuit:
Ures = qc_dec.get_matrix()

# --- Recheck the construction ---
max_abs_err = 0.0
for id_x in range(N_):
    init_state = np.zeros(N_)
    init_state[id_x] = 1

    res_U = Ures.dot(init_state)[0,id_x]
    res_dir_exp = np.exp(-1.j*t_*x[id_x]) # direct computation of the exponential function;

    abs_err = np.abs(res_U - res_dir_exp)
    if abs_err > max_abs_err:
        max_abs_err = abs_err

print("max. abs. diff: {:0.3e}".format(max_abs_err))

      ┌─────────────┐  
q_0: ─┤ Rz(-3.1447) ├──
      ├─────────────┤  
q_1: ─┤ Rz(-1.5723) ├──
      ├─────────────┴┐ 
q_2: ─┤ Rz(-0.78617) ├─
      ├──────────────┤ 
q_3: ─┤ Rz(-0.39308) ├─
      ├──────────────┤ 
q_4: ─┤ Rz(-0.19654) ├─
     ┌┴──────────────┤ 
q_5: ┤ Rz(-0.098271) ├─
     ├───────────────┤ 
q_6: ┤ Rz(-0.049135) ├─
     ├───────────────┤ 
q_7: ┤ Rz(-0.024568) ├─
     ├───────────────┤ 
q_8: ┤ Rz(-0.012284) ├─
     ├───────────────┴┐
q_9: ┤ Rz(-0.0061419) ├
     └────────────────┘

max. abs. diff: 2.000e+00
